# Identifying Arbitrage Opportunities

This script identifies arbitrage opportunities across a set of cryptocurrency exchanges by returning the current value of any given cryptocurrency on any given exchange, and identifying the exchanges with the highest and lowest values for said cryptocurrency.

#### API 

1. 

In [7]:
import pandas as pd
import numpy as np
import requests
import json

In [21]:
exchanges = [
    "Bitfinex",
    "Binance",
    "Poloniex",
    "Coinbase",
    "BitTrex" 
]

In [18]:
def api_test(exchange, currency2):
    
    api_key = 'c32565db2e025bfaba2a25ccc2068d5aa44ddaefd51277bfd833e5649606defc'

    url = 'https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms={}&api_key={}&e={}'.format(currency2, api_key, exchange)

    headers = {
        'Content-Type': "application/json"
    }
    
    response = requests.request("GET", url, headers=headers)
    response = response.json()
    r = response

    return r

In [ ]:
def get_coin_price(currency1, currency2, exchange, api_key):
    url = 'https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms={}&api_key={}&e={}'
    .format(currency1, currency2, api_key, exchange)


In [20]:
print("Binance: ", api_test('Binance', 'USDT'))
print("Bitfinex: ", api_test('Bitfinex', 'USD'))
print("Poloniex: ", api_test('Poloniex', 'USD'))

Binance:  {'USDT': 6909.81}
Bitfinex:  {'USD': 6916.8}
Poloniex:  {'USD': 6340}


In [22]:
def find_arbitarage_opportunities(currency1, currency2, exchanges, api_key):
    
    usd_exchange = ['Coinbase', 'BitTrex', 'Poloniex', 'Bitfinex']
    usdt_exchange = ['Binance']
    exchange_list = []
    value_list = []
    
    for exchange in exchanges:
        coin_price = get_coin_price(currency1, currency2, exchange, api_key)
        if exchange in usd_exchange:
            coin_price = coin_price['USD']
        else:
            coin_price = coin_price['USDT']
        
        coin_price
        
    for exchange in exchanges:
        exchange_list.append(exchange)

Bitfinex
Binance
Poloniex
Coinbase
BitTrex


In [24]:
if 'D' in ['A', 'B', 'C']: print('yes')